<a href="https://colab.research.google.com/github/Tydos/Pharmaceutical-Supply-Chain-Optimisation/blob/main/notebook/Supply_Chain_Optimisation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Supply Chain Optimisation

Predicting the optimal shipment mode given certain factors

In [111]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [112]:
import pandas as pd
import seaborn as sns

### Load the dataset

The PEPFAR Supply Chain Management System (SCMS) Delivery History dataset contains detailed, shipment-level records for antiretroviral (ARV) and HIV laboratory testing kits delivered to various countries. This data is primarily used for analyzing logistics performance, such as on-time delivery percentages, delivery delays, and costs.

In [122]:
file_path = "/content/drive/MyDrive/supply_chain.csv"
try:
  df = pd.read_csv(file_path,encoding='latin1')
except Exception as e:
  print(e)


In [123]:
df.sample(1)

,ID,Project Code,PQ #,PO / SO #,ASN/DN #,Country,Managed By,Fulfill Via,Vendor INCO Term,Shipment Mode,...,Unit of Measure (Per Pack),Line Item Quantity,Line Item Value,Pack Price,Unit Price,Manufacturing Site,First Line Designation,Weight (Kilograms),Freight Cost (USD),Line Item Insurance (USD)
5844,69235,116-ZA-T30,FPQ-13973,SCMS-238552,ASN-27609,South Africa,PMO - US,Direct Drop,DDP,Truck,...,120,24,850.06,35.42,0.3,"Janssen-Cilag, Latina, IT",No,Weight Captured Separately,Freight Included in Commodity Cost,0.87


In [124]:
cat_cols = df.select_dtypes(include=['object']).columns.to_list()
num_cols = df.select_dtypes(include=['int64','float64']).columns.to_list()
print(num_cols)
print(cat_cols)

['ID', 'Unit of Measure (Per Pack)', 'Line Item Quantity', 'Line Item Value', 'Pack Price', 'Unit Price', 'Line Item Insurance (USD)']
['Project Code', 'PQ #', 'PO / SO #', 'ASN/DN #', 'Country', 'Managed By', 'Fulfill Via', 'Vendor INCO Term', 'Shipment Mode', 'PQ First Sent to Client Date', 'PO Sent to Vendor Date', 'Scheduled Delivery Date', 'Delivered to Client Date', 'Delivery Recorded Date', 'Product Group', 'Sub Classification', 'Vendor', 'Item Description', 'Molecule/Test Type', 'Brand', 'Dosage', 'Dosage Form', 'Manufacturing Site', 'First Line Designation', 'Weight (Kilograms)', 'Freight Cost (USD)']


### Create the training, validation and test sets

In [125]:
from sklearn.model_selection import train_test_split
X = df.drop(columns=["Shipment Mode"],axis=1)
y = df["Shipment Mode"]
X_holdout, X_test, y_holdout, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=10)

X_train, X_val, y_train, y_val = train_test_split(
    X_holdout,
    y_holdout,
    test_size=0.3,
    random_state=10
)


In [126]:
print("length of dataset:",len(df))
print("length of train set:",len(X_train))
print("length of val set:",len(X_val))
print("length of test set:",len(X_test))

length of dataset: 10324
length of train set: 5781
length of val set: 2478
length of test set: 2065


### Feature Engineering

In [127]:
# drop irrelevant features
DROP_COLS = ["ID",  "Project Code",
    "PQ #",
    "PO / SO #",
    "ASN/DN #",]

X_train = X_train.drop(columns=DROP_COLS)
X_val   = X_val.drop(columns=DROP_COLS)
X_test  = X_test.drop(columns=DROP_COLS)

In [129]:
# engineer date based features
# converted date-based columsn to strictly date, and engineered two features combining the exisiting features
date_cols = [
    "PQ First Sent to Client Date",
    "PO Sent to Vendor Date",
    "Scheduled Delivery Date",
    "Delivered to Client Date",
    "Delivery Recorded Date"

]

for col in date_cols:
    X_train[col] = pd.to_datetime(X_train[col], errors="coerce")
    X_val[col]   = pd.to_datetime(X_val[col], errors="coerce")
    X_test[col]  = pd.to_datetime(X_test[col], errors="coerce")

for df in [X_train, X_val, X_test]:
    df["processing_time"] = (
        df_["PO Sent to Vendor Date"] - df["PQ First Sent to Client Date"]
    ).dt.days

    df["lead_time_requested"] = (
        df["Scheduled Delivery Date"] - df["PO Sent to Vendor Date"]
    ).dt.days

    df["delivery_lateness_days"] = (
    df["Delivered to Client Date"] - df["Scheduled Delivery Date"]
    ).dt.days

    df["recording_delay_days"] = (
    df["Delivery Recorded Date"] - df["Delivered to Client Date"]
    ).dt.days

X_train.drop(columns=date_cols, inplace=True)
X_val.drop(columns=date_cols, inplace=True)
X_test.drop(columns=date_cols, inplace=True)

In [131]:
X_train

,Country,Managed By,Fulfill Via,Vendor INCO Term,Product Group,Sub Classification,Vendor,Item Description,Molecule/Test Type,Brand,...,Unit Price,Manufacturing Site,First Line Designation,Weight (Kilograms),Freight Cost (USD),Line Item Insurance (USD),processing_time,lead_time_requested,delivery_lateness_days,recording_delay_days
10126,Uganda,PMO - US,From RDC,N/A - From RDC,ARV,Adult,SCMS from RDC,"Lopinavir/Ritonavir 200/50mg, tablets, 120 Tabs",Lopinavir/Ritonavir,Generic,...,0.22,"Aurobindo Unit III, India",Yes,2034,10977.65,275.29,NaN,NaN,-9,0
2646,Nigeria,PMO - US,From RDC,N/A - From RDC,ARV,Adult,SCMS from RDC,"Nevirapine 200mg, tablets, 60 Tabs",Nevirapine,Generic,...,0.05,Hetero Unit III Hyderabad IN,Yes,60,1981.1,6.69,NaN,NaN,0,0
2785,Haiti,PMO - US,Direct Drop,EXW,ARV,Pediatric,Aurobindo Pharma Limited,"Efavirenz 200mg, capsule, 90 Caps",Efavirenz,Generic,...,0.06,"Aurobindo Unit III, India",Yes,6385,8768.76,25.04,40.0,149.0,0,0
7208,Zambia,PMO - US,From RDC,N/A - From RDC,ARV,Pediatric,SCMS from RDC,"Nevirapine 10mg/ml, oral suspension w/syringe,...",Nevirapine,Generic,...,0.01,"Aurobindo Unit III, India",Yes,1487,5447.21,12.91,NaN,NaN,-32,0
8437,Côte d'Ivoire,PMO - US,From RDC,N/A - From RDC,ARV,Pediatric,SCMS from RDC,"Nevirapine 10mg/ml, oral suspension, Bottle, 2...",Nevirapine,Generic,...,0.01,"Aurobindo Unit III, India",No,See DN-1662 (ID#:83981),See DN-1662 (ID#:83981),5.39,NaN,NaN,-8,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8879,Zambia,PMO - US,From RDC,N/A - From RDC,ARV,Adult,SCMS from RDC,"Nevirapine 200mg, tablets, 60 Tabs",Nevirapine,Generic,...,0.04,"Strides, Bangalore, India.",No,Weight Captured Separately,Freight Included in Commodity Cost,200.74,NaN,NaN,10,9
2240,Haiti,PMO - US,From RDC,N/A - From RDC,ARV,Adult,SCMS from RDC,"Efavirenz 600mg, tablets, 30 Tabs",Efavirenz,Generic,...,0.32,"Aurobindo Unit III, India",No,See DN-1221 (ID#:11636),See DN-1221 (ID#:11636),191.75,NaN,NaN,0,0
10224,"Congo, DRC",PMO - US,From RDC,N/A - From RDC,ARV,Adult,SCMS from RDC,"Efavirenz 600mg, tablets, 30 Tabs",Efavirenz,Generic,...,0.11,"Strides, Bangalore, India.",No,See DN-3561 (ID#:82654),See DN-3561 (ID#:82654),9.68,NaN,NaN,-11,38
4483,South Sudan,PMO - US,Direct Drop,EXW,HRDT,HIV test,"Orgenics, Ltd","HIV 1/2, Determine Complete HIV Kit, 100 Tests","HIV 1/2, Determine Complete HIV Kit",Determine,...,0.89,"Alere Medical Co., Ltd.",Yes,188,2997.85,49.54,18.0,95.0,0,3


#### INCOTERM

| INCOTERM | Full Name                   | Main Responsibility Holder   | Risk Transfer Point | Typical Shipment Mode Bias | ML Interpretation           |
| -------- | --------------------------- | ---------------------------- | ------------------- | -------------------------- | --------------------------- |
| **EXW**  | Ex Works                    | Buyer                        | At vendor premises  | Sea / Road                 | Cost-driven, low urgency    |
| **FCA**  | Free Carrier                | Buyer (after handoff)        | At first carrier    | Road / Air                 | Flexible, buyer-optimized   |
| **FOB**  | Free on Board               | Vendor (until vessel)        | On vessel           | Sea                        | Bulk, non-urgent            |
| **CFR**  | Cost and Freight            | Vendor (freight paid)        | Destination port    | Sea                        | Cost absorbed, low urgency  |
| **CIF**  | Cost, Insurance, Freight    | Vendor (freight + insurance) | Destination port    | Sea                        | Risk-managed bulk           |
| **DAP**  | Delivered at Place          | Vendor                       | At destination      | Air / Express Road         | High service level          |
| **DPU**  | Delivered at Place Unloaded | Vendor                       | After unloading     | Air                        | Reliability focused         |
| **DDP**  | Delivered Duty Paid         | Vendor (end-to-end)          | Final delivery      | Air / Cold-chain Express   | Compliance & speed critical |


#### Location Extraction

### Encoding

In [ ]:
from sklearn.preprocessing import LabelEncoder

### Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score